In [1]:
import featuretools as ft

In [2]:
data = ft.demo.load_mock_customer()

transactions_df = data["transactions"].merge(data["sessions"]).merge(data["customers"])

transactions_df.sample(10)

,transaction_id,session_id,transaction_time,product_id,amount,customer_id,device,session_start,zip_code,join_date,date_of_birth
264,380,21,2014-01-01 05:14:10,5,57.09,4,desktop,2014-01-01 05:02:15,60091,2011-04-08 20:08:14,2006-08-15
19,244,10,2014-01-01 02:34:55,2,116.95,2,tablet,2014-01-01 02:31:40,13244,2012-04-15 23:31:04,1986-08-18
314,299,6,2014-01-01 01:32:05,4,64.99,1,tablet,2014-01-01 01:23:25,60091,2011-04-17 10:48:33,1994-07-18
290,78,4,2014-01-01 00:54:10,1,37.50,1,mobile,2014-01-01 00:44:25,60091,2011-04-17 10:48:33,1994-07-18
379,457,27,2014-01-01 06:37:35,1,19.16,1,mobile,2014-01-01 06:34:20,60091,2011-04-17 10:48:33,1994-07-18
335,477,9,2014-01-01 02:30:35,3,41.70,1,desktop,2014-01-01 02:15:25,60091,2011-04-17 10:48:33,1994-07-18
293,103,4,2014-01-01 00:57:25,5,20.79,1,mobile,2014-01-01 00:44:25,60091,2011-04-17 10:48:33,1994-07-18
271,390,22,2014-01-01 05:21:45,2,54.83,4,desktop,2014-01-01 05:21:45,60091,2011-04-08 20:08:14,2006-08-15
404,476,29,2014-01-01 07:24:10,4,121.59,1,mobile,2014-01-01 07:10:05,60091,2011-04-17 10:48:33,1994-07-18
179,90,3,2014-01-01 00:35:45,1,75.73,4,mobile,2014-01-01 00:28:10,60091,2011-04-08 20:08:14,2006-08-15


In [4]:
products_df = data["products"]
products_df.head()

,product_id,brand
0,1,B
1,2,B
2,3,B
3,4,B
4,5,A


In [5]:
es = ft.EntitySet(id="customer_data")

In [7]:
es = es.entity_from_dataframe(entity_id="transactions",
                              dataframe=transactions_df,
                              index="transaction_id",
                              time_index="transaction_time",
                              variable_types={"product_id": ft.variable_types.Categorical,
                                              "zip_code": ft.variable_types.ZIPCode})
es

Entityset: customer_data
  Entities:
    transactions [Rows: 500, Columns: 11]
  Relationships:
    No relationships

In [8]:
es = es.entity_from_dataframe(entity_id="products",
                              dataframe=products_df,
                              index="product_id")
es

Entityset: customer_data
  Entities:
    transactions [Rows: 500, Columns: 11]
    products [Rows: 5, Columns: 2]
  Relationships:
    No relationships

In [9]:
new_relationship = ft.Relationship(es["products"]["product_id"],
                                   es["transactions"]["product_id"])

In [10]:
es = es.add_relationship(new_relationship)

In [11]:
es

Entityset: customer_data
  Entities:
    transactions [Rows: 500, Columns: 11]
    products [Rows: 5, Columns: 2]
  Relationships:
    transactions.product_id -> products.product_id

In [12]:
feature_matrix, feature_defs = ft.dfs(entityset=es,
                                      target_entity="products")

In [13]:
feature_matrix

,brand,SUM(transactions.session_id),SUM(transactions.amount),SUM(transactions.customer_id),STD(transactions.session_id),STD(transactions.amount),STD(transactions.customer_id),MAX(transactions.session_id),MAX(transactions.amount),MAX(transactions.customer_id),...,MODE(transactions.YEAR(join_date)),MODE(transactions.YEAR(date_of_birth)),MODE(transactions.MONTH(transaction_time)),MODE(transactions.MONTH(session_start)),MODE(transactions.MONTH(join_date)),MODE(transactions.MONTH(date_of_birth)),MODE(transactions.WEEKDAY(transaction_time)),MODE(transactions.WEEKDAY(session_start)),MODE(transactions.WEEKDAY(join_date)),MODE(transactions.WEEKDAY(date_of_birth))
product_id,,,,,,,,,,,,,,,,,,,,,
1,B,1809,7489.79,298,10.640783,42.479989,1.404986,35,149.56,5,...,2011,1994,1,1,4,7,2,2,5,0
2,B,1571,7021.43,271,9.391642,46.336308,1.424774,34,149.95,5,...,2011,2006,1,1,4,8,2,2,6,0
3,B,1832,7008.12,291,10.703336,38.871405,1.341273,35,148.31,5,...,2011,2006,1,1,4,8,2,2,5,0
4,B,1894,8088.97,273,9.942765,42.492501,1.440556,35,146.46,5,...,2011,1994,1,1,4,7,2,2,6,0
5,A,1914,7931.55,289,11.129005,42.131902,1.474421,35,149.02,5,...,2011,1994,1,1,4,7,2,2,6,0
